In [2]:
import pandas as pd
import os
import geopandas as gpd
import glob
import plotly.express as px
import plotly.graph_objects as go
import folium
import json
import math
import numpy as np
import json
import requests

os.chdir("/home/ubuntu/realestate")
print(os.getcwd())

/home/ubuntu/realestate


## 데이터 로드

- 출처
  - 공공 데이터 포털(아파트 전월세):https://www.data.go.kr/data/15058017/openapi.do
  - 공공 데이터 포털(아파트 매매 실거래가):https://www.data.go.kr/iim/api/selectAPIAcountView.do

In [14]:
AptRent_Total = pd.read_csv("data/AptRent_Total.csv")
AptTrade_Total = pd.read_csv("data/AptTrade_Total.csv")

regal_code = pd.read_csv('data/regal_code.csv')
regal_code = regal_code.rename(columns={"법정동시군구코드":"지역코드"})
regal_code = regal_code.where(pd.notnull(regal_code), " ")

regal_code['시도명'] = regal_code['시도명'].str.strip()
regal_code['시군구명'] = regal_code['시군구명'].str.strip()
regal_code['읍면동명'] = regal_code['읍면동명'].str.strip()
regal_code['동리명'] = regal_code['동리명'].str.strip()

regal_code['주소'] = regal_code['시도명'] + " " + \
                       regal_code['시군구명'] + " " + \
                       regal_code['읍면동명'] + " " + \
                       regal_code['동리명']

regal_code['주소'] = regal_code['주소'].str.replace('  ', ' ')
regal_code['주소'] = regal_code['주소'].str.strip()
regal_code

,법정동코드,시도명,시군구명,읍면동명,동리명,생성일자,말소일자,지역코드,법정동읍면동코드,주소
0,1111010100,서울특별시,종로구,청운동,,19880423,,11110,10100,서울특별시 종로구 청운동
1,1111010200,서울특별시,종로구,신교동,,19880423,,11110,10200,서울특별시 종로구 신교동
2,1111010300,서울특별시,종로구,궁정동,,19880423,,11110,10300,서울특별시 종로구 궁정동
3,1111010400,서울특별시,종로구,효자동,,19880423,,11110,10400,서울특별시 종로구 효자동
4,1111010500,서울특별시,종로구,창성동,,19880423,,11110,10500,서울특별시 종로구 창성동
...,...,...,...,...,...,...,...,...,...,...
20273,5013032022,제주특별자치도,서귀포시,표선면,하천리,20060701,,50130,32022,제주특별자치도 서귀포시 표선면 하천리
20274,5013032023,제주특별자치도,서귀포시,표선면,성읍리,20060701,,50130,32023,제주특별자치도 서귀포시 표선면 성읍리
20275,5013032024,제주특별자치도,서귀포시,표선면,가시리,20060701,,50130,32024,제주특별자치도 서귀포시 표선면 가시리
20276,5013032025,제주특별자치도,서귀포시,표선면,세화리,20060701,,50130,32025,제주특별자치도 서귀포시 표선면 세화리



#### 가설 리스트


- 매매/전세가 현황 조사
- 하락장 분위기 속 구별되게 상승하는 지역
- 상승 지역 상승 원인 조사
- 주변 지역 호재 (재개발, 인근 분양 등)


## 아파트 전월세

In [6]:
df1 = pd.merge(df, regal_code[['지역코드', '시군구명','주소']],
        on = '지역코드',
        how = 'left')

In [7]:
df1

,갱신요구권사용,건축년도,계약구분,계약기간,년,법정동,보증금액,아파트,월,월세금액,일,전용면적,종전계약보증금,종전계약월세,지번,지역코드,층,시군구명
0,,2000.0,,,2020,청운동,10000,청운현대,1,130,6,84.51,,,56-45,11110,2,종로구
1,,2000.0,,,2020,청운동,10000,청운현대,1,130,6,84.51,,,56-45,11110,2,종로구
2,,2000.0,,,2020,청운동,10000,청운현대,1,130,6,84.51,,,56-45,11110,2,종로구
3,,2000.0,,,2020,청운동,10000,청운현대,1,130,6,84.51,,,56-45,11110,2,종로구
4,,2000.0,,,2020,청운동,10000,청운현대,1,130,6,84.51,,,56-45,11110,2,종로구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10102383,사용,2009.0,갱신,22.12~24.12,2022,강일동,50000,강일리버파크6단지,12,0,31,84.83,"70,000",,684,11740,7,강동구
10102384,사용,2009.0,갱신,22.12~24.12,2022,강일동,50000,강일리버파크6단지,12,0,31,84.83,"70,000",,684,11740,7,강동구
10102385,사용,2009.0,갱신,22.12~24.12,2022,강일동,50000,강일리버파크6단지,12,0,31,84.83,"70,000",,684,11740,7,강동구
10102386,사용,2009.0,갱신,22.12~24.12,2022,강일동,50000,강일리버파크6단지,12,0,31,84.83,"70,000",,684,11740,7,강동구


### 시군구별, 법정동별 거래금액 및 거래량

In [8]:
df2 = df1[['보증금액','시군구명', '법정동']].groupby(['시군구명', '법정동']).describe()

In [9]:
df2 = df2.reset_index()


In [10]:
df2.head()

시군구명  법정동      보증금액                                                       \
                count          mean           std    min      25%      50%   
0  강남구  개포동  113834.0  59539.520969  43347.595420    0.0  30000.0  48500.0   
1  강남구  논현동   32662.0  47457.694385  47898.247338  120.0   7182.0  36000.0   
2  강남구  대치동  134638.0  83796.672975  54472.666048    0.0  50000.0  71400.0   
3  강남구  도곡동   89250.0  87167.017098  60956.828942    0.0  45000.0  75100.0   
4  강남구  삼성동   64750.0  69752.467892  52230.495635    0.0  29000.0  63000.0   

                       
        75%       max  
0   81900.0  320000.0  
1   70000.0  350000.0  
2  115000.0  430000.0  
3  120000.0  500000.0  
4  100000.0  450000.0

### 시군구별, 법정동별 날짜별 거래금액 및 거래량

In [11]:
df3 = df1[['보증금액','시군구명', '법정동', '년', '월']].groupby(['시군구명', '법정동', '년', '월']).describe()

In [12]:
df3 = df3.reset_index()

In [13]:
df3['월'] = df3["월"].astype('str').str.zfill(2)

In [14]:
df3.head()

시군구명  법정동     년   월    보증금액                                               \
                        count          mean           std     min      25%   
0  강남구  개포동  2020  01  1974.0  44897.446809  31072.622043  1000.0  30000.0   
1  강남구  개포동  2020  02  2030.0  41620.413793  27380.685404  1000.0  26250.0   
2  강남구  개포동  2020  03  1526.0  47160.550459  37049.719763  1000.0  30000.0   
3  강남구  개포동  2020  04  1414.0  44080.099010  29683.736178  1000.0  27000.0   
4  강남구  개포동  2020  05  1750.0  50376.400000  30779.826399  1000.0  30000.0   

                               
       50%      75%       max  
0  41000.0  52000.0  175000.0  
1  39000.0  50000.0  160000.0  
2  41000.0  53000.0  240000.0  
3  38000.0  52000.0  145000.0  
4  42000.0  60000.0  160000.0

In [15]:
df3['보증금액']

,count,mean,std,min,25%,50%,75%,max
0,1974.0,44897.446809,31072.622043,1000.0,30000.0,41000.0,52000.0,175000.0
1,2030.0,41620.413793,27380.685404,1000.0,26250.0,39000.0,50000.0,160000.0
2,1526.0,47160.550459,37049.719763,1000.0,30000.0,41000.0,53000.0,240000.0
3,1414.0,44080.099010,29683.736178,1000.0,27000.0,38000.0,52000.0,145000.0
4,1750.0,50376.400000,30779.826399,1000.0,30000.0,42000.0,60000.0,160000.0
...,...,...,...,...,...,...,...,...
10405,132.0,31687.272727,12775.616989,1000.0,27300.0,32500.0,39900.0,55000.0
10406,108.0,28741.666667,14311.941129,1000.0,20000.0,32550.0,39000.0,52000.0
10407,114.0,25919.105263,14423.715048,1000.0,10000.0,29100.0,35000.0,49000.0
10408,138.0,29208.695652,16581.184827,1000.0,18000.0,30000.0,42000.0,56000.0


In [16]:
df3_total = pd.concat([df3[['시군구명','법정동','년','월']], df3['보증금액']], axis = 1) # 컬럼별로 붙힐때 axis = 1
df3_total.columns = ['시군구명','법정동','년','월','거래량','평균금액','표준편차','최소값','1분위수','중앙값','3분위수','최대값']

In [17]:
df3_total

,시군구명,법정동,년,월,거래량,평균금액,표준편차,최소값,1분위수,중앙값,3분위수,최대값
0,강남구,개포동,2020,01,1974.0,44897.446809,31072.622043,1000.0,30000.0,41000.0,52000.0,175000.0
1,강남구,개포동,2020,02,2030.0,41620.413793,27380.685404,1000.0,26250.0,39000.0,50000.0,160000.0
2,강남구,개포동,2020,03,1526.0,47160.550459,37049.719763,1000.0,30000.0,41000.0,53000.0,240000.0
3,강남구,개포동,2020,04,1414.0,44080.099010,29683.736178,1000.0,27000.0,38000.0,52000.0,145000.0
4,강남구,개포동,2020,05,1750.0,50376.400000,30779.826399,1000.0,30000.0,42000.0,60000.0,160000.0
...,...,...,...,...,...,...,...,...,...,...,...,...
10405,중랑구,중화동,2022,08,132.0,31687.272727,12775.616989,1000.0,27300.0,32500.0,39900.0,55000.0
10406,중랑구,중화동,2022,09,108.0,28741.666667,14311.941129,1000.0,20000.0,32550.0,39000.0,52000.0
10407,중랑구,중화동,2022,10,114.0,25919.105263,14423.715048,1000.0,10000.0,29100.0,35000.0,49000.0
10408,중랑구,중화동,2022,11,138.0,29208.695652,16581.184827,1000.0,18000.0,30000.0,42000.0,56000.0


## 아파트 매매

In [8]:
AptTrade_Total.head()

,거래금액,거래유형,건축년도,년,법정동,아파트,월,일,전용면적,중개사소재지,지번,지역코드,층,해제사유발생일,해제여부
0,100000,,2007,2020,필운동,신동아블루아광화문의꿈,1,3,111.97,,254,11110,6,,
1,162000,,2008,2020,사직동,광화문스페이스본(106동),1,2,163.33,,9-1,11110,2,,
2,88000,,1994,2020,내수동,세종로대우,1,6,109.91,,167,11110,11,,
3,130000,,2004,2020,내수동,킹스매너,1,9,194.43,,110-15,11110,4,,
4,149000,,2003,2020,내수동,경희궁파크팰리스,1,20,148.09,,95,11110,8,,


In [16]:
AptTrade_Total_2 = pd.merge(AptTrade_Total, regal_code[['지역코드', '시군구명','주소']],
                            on = '지역코드',
                            how = 'left')
AptTrade_Total_2.head()

,거래금액,거래유형,건축년도,년,법정동,아파트,월,일,전용면적,중개사소재지,지번,지역코드,층,해제사유발생일,해제여부,시군구명,주소
0,100000,,2007,2020,필운동,신동아블루아광화문의꿈,1,3,111.97,,254,11110,6,,,종로구,서울특별시 종로구 청운동
1,100000,,2007,2020,필운동,신동아블루아광화문의꿈,1,3,111.97,,254,11110,6,,,종로구,서울특별시 종로구 신교동
2,100000,,2007,2020,필운동,신동아블루아광화문의꿈,1,3,111.97,,254,11110,6,,,종로구,서울특별시 종로구 궁정동
3,100000,,2007,2020,필운동,신동아블루아광화문의꿈,1,3,111.97,,254,11110,6,,,종로구,서울특별시 종로구 효자동
4,100000,,2007,2020,필운동,신동아블루아광화문의꿈,1,3,111.97,,254,11110,6,,,종로구,서울특별시 종로구 창성동


In [19]:
AptTrade_Total_2['주소'] + ' ' +  AptTrade_Total_2['지번'] + ' ' + AptTrade_Total_2['아파트']

0          서울특별시 종로구 청운동 254 신동아블루아광화문의꿈
1          서울특별시 종로구 신교동 254 신동아블루아광화문의꿈
2          서울특별시 종로구 궁정동 254 신동아블루아광화문의꿈
3          서울특별시 종로구 효자동 254 신동아블루아광화문의꿈
4          서울특별시 종로구 창성동 254 신동아블루아광화문의꿈
                       ...              
2032313      서울특별시 강동구 둔촌동 114 강동리버스트4단지
2032314      서울특별시 강동구 암사동 114 강동리버스트4단지
2032315      서울특별시 강동구 성내동 114 강동리버스트4단지
2032316      서울특별시 강동구 천호동 114 강동리버스트4단지
2032317      서울특별시 강동구 강일동 114 강동리버스트4단지
Length: 2032318, dtype: object

### 시군구별, 법정동별 거래금액

In [20]:
AptTrade_Total_3 = AptTrade_Total_2[['거래금액','시군구명', '법정동']].groupby(['시군구명', '법정동']).describe()
AptTrade_Total_3 = AptTrade_Total_3.reset_index()

AptTrade_Total_4 = pd.concat([AptTrade_Total_3[['시군구명','법정동']], 
                              AptTrade_Total_3['거래금액']], 
                             axis = 1) # 컬럼별로 붙힐때 axis = 1
AptTrade_Total_4.columns = ['시군구명','읍면동명','거래량','평균금액','표준편차','최소값','1분위수','중앙값','3분위수','최대값']

In [21]:
AptTrade_Total_4

,시군구명,읍면동명,거래량,평균금액,표준편차,최소값,1분위수,중앙값,3분위수,최대값
0,강남구,개포동,10724.0,192221.919060,72770.455883,34300.0,131000.0,189000.0,243000.0,590000.0
1,강남구,논현동,5810.0,120023.790361,79880.522246,14200.0,41500.0,106584.0,165000.0,460000.0
2,강남구,대치동,10542.0,228098.225764,110360.864849,19000.0,185000.0,232000.0,300000.0,535000.0
3,강남구,도곡동,15092.0,231866.128015,110903.658214,23800.0,172500.0,220000.0,290000.0,870000.0
4,강남구,삼성동,7854.0,181976.226381,123111.798058,30000.0,88000.0,157000.0,241000.0,700000.0
...,...,...,...,...,...,...,...,...,...,...
327,중랑구,면목동,7098.0,54882.569738,24171.961318,8000.0,37800.0,54700.0,69000.0,149000.0
328,중랑구,묵동,4692.0,62360.383632,25338.439329,7700.0,43000.0,59500.0,79300.0,160000.0
329,중랑구,상봉동,5238.0,55550.240550,29323.095124,10000.0,32750.0,55500.0,74000.0,220000.0
330,중랑구,신내동,9228.0,54410.195059,16946.088750,18500.0,41500.0,54000.0,66800.0,113000.0


In [22]:
AptTrade_Total_2

,거래금액,거래유형,건축년도,년,법정동,아파트,월,일,전용면적,중개사소재지,지번,지역코드,층,해제사유발생일,해제여부,시군구명
0,100000,,2007,2020,필운동,신동아블루아광화문의꿈,1,3,111.97,,254,11110,6,,,종로구
1,100000,,2007,2020,필운동,신동아블루아광화문의꿈,1,3,111.97,,254,11110,6,,,종로구
2,100000,,2007,2020,필운동,신동아블루아광화문의꿈,1,3,111.97,,254,11110,6,,,종로구
3,100000,,2007,2020,필운동,신동아블루아광화문의꿈,1,3,111.97,,254,11110,6,,,종로구
4,100000,,2007,2020,필운동,신동아블루아광화문의꿈,1,3,111.97,,254,11110,6,,,종로구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2032313,55000,중개거래,2020,2022,강일동,강동리버스트4단지,12,29,59.99,서울 강동구,114,11740,7,,,강동구
2032314,55000,중개거래,2020,2022,강일동,강동리버스트4단지,12,29,59.99,서울 강동구,114,11740,7,,,강동구
2032315,55000,중개거래,2020,2022,강일동,강동리버스트4단지,12,29,59.99,서울 강동구,114,11740,7,,,강동구
2032316,55000,중개거래,2020,2022,강일동,강동리버스트4단지,12,29,59.99,서울 강동구,114,11740,7,,,강동구


In [19]:
df_3 = df_2[['거래금액','시군구명', '법정동', '년', '월']].groupby(['시군구명', '법정동', '년', '월']).describe()

KeyError: "['년' '월'] not in index"

##  주소 위경도



In [3]:
api_key = '66be80af3672f1fd2d43413af6e1588b'

def addr_to_lat_lon(addr):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query={address}'.format(address=addr)
    headers = {"Authorization": "KakaoAK " + api_key}
    result = json.loads(str(requests.get(url, headers=headers).text))
    match_first = result['documents'][0]['address']
    return float(match_first['x']), float(match_first['y'])

In [26]:
addr_to_lat_lon('서울특별시 종로구 청운동 254')

IndexError: list index out of range